## Dependecies


In [ ]:
import sys
from pathlib import Path
from torchvision import datasets, transforms
import torch.nn.functional as F
import random
import torch
import numpy as np
import json
from tqdm import tqdm, trange
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
import matplotlib.pyplot as plt

## Utils


In [ ]:
project_root = Path.cwd().resolve().parents[2]
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

data_root = project_root / "data"
data_root.mkdir(parents=True, exist_ok=True)

from notebook_setup import setup_notebook

setup_notebook()

In [ ]:
# Device configuration
from utils.utils import DEVICE

print(f"Device used: {DEVICE}")

# Set random seed for reproducibility
from utils.utils import set_seed

set_seed()

from utils.utils import save_model

# Prepare Dataloaders
from methods.naive.naive_utils import init_dataloaders

# Train loop
from utils.train_test_metrics import train_model

# Plot losses
from utils.train_test_metrics import plot_training_history

# Test function
from utils.train_test_metrics import test_model

# Merics
from utils.train_test_metrics import show_metrics

# Init model
from models.resnet50 import init_model_resnet50, load_model_resnet50

from methods.fisher.fisher_utils import iterative_fisher_unlearn

# from methods.influence.influence_utils import iterative_influence_unlearn

## Parameters (arbitrary chosen)


In [ ]:
BATCH_SIZE = 256
LEARNING_RATE = 0.001
EPOCHS = 5

# Utils


## Representative & Backup Selection


In [ ]:
def select_representative_and_backup_indices(dataset, rep_size=25000):
    """
    Randomly select rep_size indices from the dataset as the representative subset,
    and treat the remaining indices as the backup set.
    """
    total_indices = list(range(len(dataset)))
    rep_indices = random.sample(total_indices, rep_size)
    backup_indices = list(set(total_indices) - set(rep_indices))
    return rep_indices, backup_indices

## Unlearn Sample Selection (for target class)


In [ ]:
def select_class_samples(dataset, indices, target_class, num_samples=25):
    """
    From the provided list of original indices (within the dataset),
    randomly select num_samples that belong to the specified target_class.
    """
    target_indices = [idx for idx in indices if dataset[idx][1] == target_class]
    if len(target_indices) < num_samples:
        raise ValueError(
            f"Not enough samples of class {target_class} available (found {len(target_indices)})."
        )
    return random.sample(target_indices, num_samples)

In [ ]:
class IndexedDataset(Dataset):
    def __init__(self, base_dataset, indices):
        self.base = base_dataset
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        orig_idx = self.indices[i]
        img, _ = self.base[orig_idx]
        return orig_idx, img

In [ ]:
from scipy.spatial.distance import cdist


def find_unique_shadow_twins_raw(
    dataset,
    backup_indices,
    unlearn_indices,
    score_type="l2",
    batch_size=256,
    device="cpu",
):
    """
    For each unlearn sample (specified by its original index in the full dataset),
    find a unique shadow twin from backup_indices based on a similarity metric computed on raw pixel vectors.

    Parameters:
        dataset (Dataset): The full dataset, where dataset[idx][0] returns an image tensor.
        backup_indices (list[int]): Indices for candidate backup samples.
        unlearn_indices (list[int]): Indices for samples to be unlearned.
        score_type (str): The metric to use: "l2" for Euclidean distance or "mahalanobis" for Mahalanobis distance.

    Returns:
        mapping (dict): A mapping from each unlearn sample original index to its unique backup index.
        similarity_metrics (dict): A mapping from each unlearn sample original index to its computed distance.
                                  (Lower values indicate higher similarity.)
    """

    # Helper: load & flatten a list of indices into a single [N × D] tensor
    def load_flat(idxs, to_device=None):
        ds = IndexedDataset(dataset, idxs)
        loader = DataLoader(
            ds, batch_size=batch_size, num_workers=4, shuffle=False, pin_memory=True
        )
        flats = []
        for _, img_batch in tqdm(loader, desc="Loading & flattening"):
            flat = img_batch.flatten(1)
            flats.append(flat.to(to_device) if to_device else flat)
        return torch.cat(flats, dim=0)

    # 1) Load & flatten UNLEARN set (small M) → GPU
    u_imgs = load_flat(unlearn_indices, to_device=device)  # [M, D]

    # 2) If Mahalanobis, precompute global whitening on ALL backups
    if score_type.lower() == "mahalanobis":
        # 2a) flatten all backups on CPU
        b_all = load_flat(backup_indices, to_device=None).double()  # [K, D] on CPU
        mu = b_all.mean(dim=0, keepdim=True)  # [1, D]
        Xc = b_all - mu
        cov = (Xc.t() @ Xc) / (b_all.size(0) - 1)  # [D, D]
        cov += 1e-5 * torch.eye(cov.size(0))  # regularize
        inv_L = torch.linalg.cholesky(torch.inverse(cov))  # [D, D] on CPU

        # Move whitening to GPU and whiten u_imgs once
        mu_dev = mu.to(device)
        inv_L_dev = inv_L.to(device)
        u_imgs = (u_imgs - mu_dev) @ inv_L_dev.T  # [M, D] ← now whitened

    # 3) Stream through backup chunks on GPU to compute distances
    ds = IndexedDataset(dataset, backup_indices)
    loader = DataLoader(
        ds, batch_size=batch_size, num_workers=4, shuffle=False, pin_memory=True
    )

    dists_chunks = []
    for _, b_img_batch in tqdm(loader, desc="Streaming BACKUP"):
        b_flat = b_img_batch.flatten(1).to(device)  # [B, D]

        if score_type.lower() == "l2":
            # Euclidean on raw pixels
            d_chunk = torch.cdist(u_imgs, b_flat, p=2)  # [M, B]

        else:
            # Mahalanobis: whiten this chunk and compute Euclid.
            b_w = (b_flat - mu_dev) @ inv_L_dev.T  # [B, D]
            d_chunk = torch.cdist(u_imgs, b_w, p=2)  # [M, B]

        dists_chunks.append(d_chunk)

    # 4) Build full [M, K] distance matrix
    dists = torch.cat(dists_chunks, dim=1)  # [M, K]

    # 5) Greedy “first-free” matching to get 25 unique twins
    mapping, similarity = {}, {}
    used = torch.zeros(dists.size(1), dtype=torch.bool, device=device)

    for i in range(dists.size(0)):
        for cand in torch.argsort(dists[i]):  # lowest‐distance first
            if not used[cand]:
                ui = unlearn_indices[i]
                bi = backup_indices[cand.item()]
                mapping[ui] = bi
                similarity[ui] = float(dists[i, cand].item())
                used[cand] = True
                break

    return mapping, similarity

In [ ]:
def find_unique_shadow_twins_features(
    model,
    dataset,
    backup_indices,
    unlearn_indices,
    score_type="cosine",  # "cosine" or "mahalanobis"
    batch_size=256,
    device="cuda:0",
):
    # 1) Unwrap & move to single GPU
    base_model = model.module if hasattr(model, "module") else model
    base_model = base_model.to(device).eval()

    # 2) Hook avgpool — overwrite output every batch
    hook_out = {}
    hook = base_model.avgpool.register_forward_hook(
        lambda module, inp, out: hook_out.update({"f": out})
    )

    # 3) Feature extractor helper
    def extract_feats(indices):
        ds = IndexedDataset(dataset, indices)
        loader = DataLoader(
            ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True
        )
        idxs_list, feats_list = [], []
        with torch.no_grad():
            for idx_batch, img_batch in tqdm(loader, desc="Extracting features"):
                imgs = img_batch.to(device, non_blocking=True)
                _ = base_model(imgs)
                feats = hook_out["f"].flatten(1)  # [B, F]
                idxs_list.append(idx_batch.to(device))
                feats_list.append(feats)
        idxs = torch.cat(idxs_list, dim=0)
        feats = torch.cat(feats_list, dim=0)
        return idxs, feats

    # 4) Extract unlearn & backup features
    u_idx, u_feat = extract_feats(unlearn_indices)  # [M], [M, F]
    b_idx, b_feat = extract_feats(backup_indices)  # [K], [K, F]
    hook.remove()

    # 5) Compute similarity/distance matrix
    if score_type == "cosine":
        u_n = F.normalize(u_feat, dim=1)  # [M, F]
        b_n = F.normalize(b_feat, dim=1)  # [K, F]
        matrix = u_n @ b_n.t()  # higher = more similar
        higher_is_better = True

    elif score_type == "mahalanobis":
        # Cast to double for stable covariance
        b_d = b_feat.double()
        mean_d = b_d.mean(0, keepdim=True)  # [1, F]
        Xc = b_d - mean_d
        cov = (Xc.t() @ Xc) / (b_d.size(0) - 1)
        cov += 1e-5 * torch.eye(cov.size(0), device=device, dtype=torch.double)
        inv_L_d = torch.linalg.cholesky(torch.inverse(cov))
        inv_L = inv_L_d.to(torch.float32)

        # Whiten features
        mean_f = mean_d.to(torch.float32)
        u_w = (u_feat - mean_f) @ inv_L.t()  # [M, F]
        b_w = (b_feat - mean_f) @ inv_L.t()  # [K, F]
        matrix = torch.cdist(u_w, b_w, p=2)  # lower = more similar
        higher_is_better = False

    else:
        raise ValueError("score_type must be 'cosine' or 'mahalanobis'")

    # 6) Greedy “first-free” unique matching
    M, K = matrix.shape
    used = torch.zeros(K, dtype=torch.bool, device=device)
    mapping, similarity = {}, {}
    for i in range(M):
        row = matrix[i]
        order = torch.argsort(-row) if higher_is_better else torch.argsort(row)
        for c in order:
            idx_c = c.item()
            if not used[idx_c]:
                ui = u_idx[i].item()
                bi = b_idx[idx_c].item()
                mapping[ui] = bi
                similarity[ui] = float(row[idx_c].item())
                used[idx_c] = True
                break
    torch.set_grad_enabled(True)
    return mapping, similarity

# Simple base training


## Init model


In [ ]:
model, model_name, criterion, optimizer, transform = init_model_resnet50(
    learning_rate=LEARNING_RATE
)
model_name = "base"

In [ ]:
model = nn.DataParallel(model, device_ids=[0, 1])

## Get data


In [ ]:
raw_transform = transforms.ToTensor()
cifar_raw = datasets.CIFAR10(
    root=data_root, train=True, download=True, transform=raw_transform
)

In [ ]:
train_dataset_full = datasets.CIFAR10(
    root=data_root, train=True, transform=transform, download=True
)
test_dataset = datasets.CIFAR10(
    root=data_root, train=False, transform=transform, download=True
)
data_split_path = "cifar10_data_splits.json"

## Create Representative & Backup Splits


In [ ]:
rep_indices, backup_indices = select_representative_and_backup_indices(
    train_dataset_full, rep_size=35000
)
print(
    f"Representative subset size: {len(rep_indices)}; Backup subset size: {len(backup_indices)}"
)

## Create representative subset based on the selected indices


In [ ]:
rep_subset = Subset(train_dataset_full, rep_indices)

## Init dataloders


In [ ]:
train_loader, valid_loader, test_loader, classes = init_dataloaders(
    datasets=(rep_subset, test_dataset), val_ratio=0.2, batch_size=BATCH_SIZE
)

## Call train


In [ ]:
# import time
# start_time = time.perf_counter()
# train_model(model, model_name, train_loader, valid_loader, criterion, optimizer, num_epochs=EPOCHS)
# end_time = time.perf_counter()  # End timer
# elapsed_time = end_time - start_time

# print(f"Execution time: {elapsed_time:.6f} seconds")

## Plot history losses


In [ ]:
# history_path = f'{model_name}_history.json'
# plot_training_history(history_path)

## Call test


In [ ]:
# model_path = f"{model_name}_model.pth"
model_path = f"/kaggle/input/ul-resnet/pytorch/default/3/base_model.pth"
test_model(model, model_name, model_path, test_loader)

## Show metrics


In [ ]:
predictions_path = f"{model_name}_predictions.json"
# classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
show_metrics(predictions_path, classes, model_name)

# Pick samples to unlearn & their shadow twins


In [ ]:
TARGET_CLASS = 4  # deer

In [ ]:
print(train_dataset_full.classes)

In [ ]:
filtered_backup_indices = [
    idx for idx in backup_indices if train_dataset_full[idx][1] == TARGET_CLASS
]

## Use the full dataset with rep_indices to select unlearn samples.


In [ ]:
unlearn_original_indices = select_class_samples(
    train_dataset_full, rep_indices, TARGET_CLASS, num_samples=25
)
print(
    f"Selected unlearn sample original indices (from full dataset): {unlearn_original_indices}"
)

## Convert these original indices to indices relative to the representative subset.


In [ ]:
unlearn_relative_indices = [rep_indices.index(u) for u in unlearn_original_indices]
print(
    f"Unlearn sample indices relative to representative subset: {unlearn_relative_indices}"
)

## Unique Shadow Twin Matching


In [ ]:
import time
import json
import csv


def save_shadow_mapping(mapping, similarity, json_path, csv_path):
    with open(json_path, "w") as f:
        json.dump(mapping, f, indent=4)
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["unlearn_idx", "backup_idx", "score"])
        for un, back in mapping.items():
            writer.writerow([un, back, similarity[un]])
    print(f"Saved JSON → {json_path} and CSV → {csv_path}")

In [ ]:
twin_mappings_list = []

# RAW L2
start_time = time.perf_counter()
raw_l2_mapping, raw_l2_similarity = find_unique_shadow_twins_raw(
    cifar_raw,
    filtered_backup_indices,
    unlearn_original_indices,
    score_type="l2",
    batch_size=128,
)
end_time = time.perf_counter()  # End timer
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")
save_shadow_mapping(
    raw_l2_mapping, raw_l2_similarity, "raw_l2_mapping.json", "raw_l2_similarity.csv"
)
twin_mappings_list.append(("raw_l2", raw_l2_mapping, raw_l2_similarity))

# RAW MAHALANOBIS
start_time = time.perf_counter()
raw_mahalanobis_mapping, raw_mahalanobis_similarity = find_unique_shadow_twins_raw(
    cifar_raw,
    filtered_backup_indices,
    unlearn_original_indices,
    score_type="mahalanobis",
    batch_size=128,
)
end_time = time.perf_counter()  # End timer
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")
save_shadow_mapping(
    raw_mahalanobis_mapping,
    raw_mahalanobis_similarity,
    "raw_mahalanobis_mapping.json",
    "raw_mahalanobis_similarity.csv",
)
twin_mappings_list.append(
    ("raw_mahalanobis", raw_mahalanobis_mapping, raw_mahalanobis_similarity)
)

In [ ]:
# FEATURE COSINE
start_time = time.perf_counter()
features_cosine_mapping, features_cosine_similarity = find_unique_shadow_twins_features(
    model,
    train_dataset_full,
    filtered_backup_indices,
    unlearn_original_indices,
    score_type="cosine",
    batch_size=128,
)
end_time = time.perf_counter()  # End timer
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")
save_shadow_mapping(
    features_cosine_mapping,
    features_cosine_similarity,
    "feat_cosine_mapping.json",
    "feat_cosine_similarity.csv",
)
twin_mappings_list.append(
    ("features_cosine", features_cosine_mapping, features_cosine_similarity)
)

# FEATURE MAHALANOBIS
start_time = time.perf_counter()
features_mahalanobis_mapping, features_mahalanobis_similarity = (
    find_unique_shadow_twins_features(
        model,
        train_dataset_full,
        filtered_backup_indices,
        unlearn_original_indices,
        score_type="mahalanobis",
        batch_size=128,
    )
)
end_time = time.perf_counter()  # End timer
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")
save_shadow_mapping(
    features_mahalanobis_mapping,
    features_mahalanobis_similarity,
    "feat_mahalanobis_mapping.json",
    "feat_mahalanobis_similarity.csv",
)
twin_mappings_list.append(
    (
        "features_mahalanobis",
        features_mahalanobis_mapping,
        features_mahalanobis_similarity,
    )
)

In [ ]:
import os

output_dir = "twin_mappings_plots"
os.makedirs(output_dir, exist_ok=True)
for method_name, mapping, _ in twin_mappings_list:
    pairs_to_show = list(mapping.items())[:15]
    rows = 5
    pairs_per_row = 3
    cols = pairs_per_row * 2  # two columns per pair (Original + Replacement)

    fig, axes = plt.subplots(rows, cols, figsize=(12, 10))
    fig.suptitle(method_name, fontsize=16)
    axes = axes.reshape(rows, cols)

    for i, (a_idx, b_idx) in enumerate(pairs_to_show):
        row = i // pairs_per_row
        col_offset = (i % pairs_per_row) * 2

        # Plot Original
        ax = axes[row, col_offset]
        ax.imshow(train_dataset_full.data[a_idx])
        ax.set_title(f"Original: {a_idx}", fontsize=8)
        ax.axis("off")

        # Plot Replacement
        ax = axes[row, col_offset + 1]
        ax.imshow(train_dataset_full.data[b_idx])
        ax.set_title(f"Replacement: {b_idx}", fontsize=8)
        ax.axis("off")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Save figure to file
    filename = f"{method_name.replace(' ', '_')}.png"
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, dpi=300, bbox_inches="tight")
    print(f"Saved figure for {method_name} to {filepath}")
    plt.show()

In [ ]:
import torch

torch.cuda.empty_cache()

import gc

gc.collect()

# Gold naive retraining


## Init new model


In [ ]:
gold_model, gold_model_name, criterion, optimizer, transform = init_model_resnet50(
    learning_rate=LEARNING_RATE
)
gold_model_name = "gold"
gold_model = nn.DataParallel(gold_model, device_ids=[0, 1])

## Get clean gold subset w/o samples to delete


In [ ]:
gold_rep_indices = sorted(list(set(rep_indices) - set(unlearn_original_indices)))
print(f"Clean representative indices count: {len(gold_rep_indices)}")
gold_subset = Subset(train_dataset_full, gold_rep_indices)

## Init dataloaders


In [ ]:
gold_train_loader, gold_valid_loader, gold_test_loader, classes = init_dataloaders(
    datasets=(gold_subset, test_dataset), val_ratio=0.2, batch_size=BATCH_SIZE
)

## Call train


In [ ]:
# import time
# start_time = time.perf_counter()
# train_model(gold_model, gold_model_name, gold_train_loader, gold_valid_loader, criterion, optimizer, num_epochs=EPOCHS)
# end_time = time.perf_counter()  # End timer
# elapsed_time = end_time - start_time

# print(f"Execution time: {elapsed_time:.6f} seconds")

## Plot history losses


In [ ]:
# history_path = f'{gold_model_name}_history.json'
# plot_training_history(history_path)

## Call test


In [ ]:
# gold_model_path = f"{gold_model_name}_model.pth"
gold_model_path = f"/kaggle/input/ul-resnet/pytorch/default/3/gold_model.pth"
test_model(gold_model, gold_model_name, gold_model_path, gold_test_loader)

## Show metrics


In [ ]:
predictions_path = f"{gold_model_name}_predictions.json"
# classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
show_metrics(predictions_path, classes, model_name)

# Unlearning


## Init new model


In [ ]:
def load_model_resnet50(model_pth_path, parallel=False):
    print("Load model...")

    model, model_name, criterion, optimizer, transform = init_model_resnet50()
    if parallel:
        model = nn.DataParallel(model, device_ids=[0, 1])

    model.load_state_dict(
        torch.load(model_pth_path, weights_only=True, map_location=DEVICE)
    )

    print("Done loading model.")

    return model, model_name, criterion, optimizer, transform

In [ ]:
# model_file = f"{model_name}_model.pth"
model_file = f"/kaggle/input/ul-resnet/pytorch/default/3/base_model.pth"
original_model, original_model_name, criterion, _optimizer, transform = (
    load_model_resnet50(model_pth_path=model_file, parallel=True)
)

## Unlearning via fisher


In [ ]:
import math
import torch
from utils.utils import DEVICE
from tqdm import tqdm
from torch.utils.data import DataLoader, Subset


def compute_gradient_on_subset(model, criterion, dataset_subset, batch_size):
    """
    Compute the average gradient Δ_rem = ∇L(θ, D') over the given dataset_subset.
    """
    dataloader = DataLoader(dataset_subset, batch_size=batch_size, shuffle=False)

    grad_dict = {}
    total_samples = 0

    model.eval()
    for inputs, targets in tqdm(dataloader, desc="Computing gradients"):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        model.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        current_batch = inputs.size(0)
        total_samples += current_batch
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                if name not in grad_dict:
                    grad_dict[name] = param.grad.detach().clone() * current_batch
                else:
                    grad_dict[name] += param.grad.detach() * current_batch

    # Average gradients over the entire subset
    for name in grad_dict:
        grad_dict[name] /= total_samples

    ## DIAGNOSTIC: Print total number of samples used for averaging
    print(f"Computed gradients on {total_samples} samples.")

    return grad_dict


def compute_fisher_on_subset(model, criterion, dataset_subset, batch_size):
    """
    Compute a diagonal approximation of the Fisher Information Matrix F over the given dataset_subset.
    It averages the squared gradients.
    """
    dataloader = DataLoader(dataset_subset, batch_size=batch_size, shuffle=False)
    fisher_diag = {}
    total_samples = 0

    model.eval()
    for inputs, targets in tqdm(dataloader, desc="Computing Fisher"):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        model.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        current_batch = inputs.size(0)
        total_samples += current_batch
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                if name not in fisher_diag:
                    fisher_diag[name] = (param.grad.detach() ** 2) * current_batch
                else:
                    fisher_diag[name] += (param.grad.detach() ** 2) * current_batch

    for name in fisher_diag:
        fisher_diag[name] /= total_samples

        ## DIAGNOSTIC: Print Fisher diagonal statistics to verify damping requirements
        print(
            f"Fisher diag for {name}: min = {fisher_diag[name].min().item():.2e}, "
            f"mean = {fisher_diag[name].mean().item():.2e}, max = {fisher_diag[name].max().item():.2e}"
        )

    return fisher_diag

In [ ]:
def iterative_fisher_unlearn_simplified_golatkar(
    model,
    criterion,
    full_dataset,
    removal_indices,
    sigma_lambda_term,
    deletion_batch_size,
    compute_batch_size,
    eps=1e-5,
):

    full_size = len(full_dataset)
    current_indices = set(range(full_size))

    removal_list = list(removal_indices)
    num_batches = math.ceil(len(removal_list) / deletion_batch_size)
    partitioned_removals = [
        removal_list[i * deletion_batch_size : (i + 1) * deletion_batch_size]
        for i in range(num_batches)
    ]
    print(f"Applying simplified Golatkar Fisher (Iterative Noise Injection)...")
    print(
        f"Total deletion samples: {len(removal_list)}; partitioned into {num_batches} mini-batches (each up to {deletion_batch_size} samples)."
    )
    print(f"Noise scale term (λσ_h): {sigma_lambda_term}")

    model.to(DEVICE)
    for i, batch in enumerate(
        tqdm(partitioned_removals, desc="Iterative Fisher Noise Injection")
    ):
        current_indices -= set(batch)
        updated_indices = sorted(list(current_indices))
        dataset_remaining = Subset(full_dataset, updated_indices)
        print(
            f"\nIteration {i+1}/{num_batches}: Remaining dataset size = {len(dataset_remaining)}"
        )

        model.eval()
        fisher_diag = compute_fisher_on_subset(
            model, criterion, dataset_remaining, compute_batch_size
        )

        model.train()
        with torch.no_grad():
            total_noise_norm_iter = 0.0
            for name, param in model.named_parameters():
                if param.requires_grad and name in fisher_diag:

                    # Calculate F^(-1/4) with damping
                    inv_fisher_quarter = (fisher_diag[name].to(DEVICE) + eps).pow(-0.25)

                    # Generate Gaussian noise n ~ N(0, I)
                    noise = torch.randn_like(param.data)

                    # Calculate the noise term: (λσ_h) * F^(-1/4) *
                    scaled_noise = sigma_lambda_term * inv_fisher_quarter * noise
                    noise_norm = scaled_noise.norm(2).item()
                    total_noise_norm_iter += noise_norm**2

                    param.data.add_(scaled_noise)

        print(
            f"Iteration {i+1}: Total added noise norm (sqrt sum sq) = {total_noise_norm_iter**0.5:.4e}"
        )
        print(f"Iteration {i+1}/{num_batches} noise injection completed.")
    return model

In [ ]:
COMPUTE_BATCH_SIZE = 128
MINI_BATCH_SIZE = 25

SIGMA = 5e-5

EPS = 1e-6
MAX_NORM = 10

import copy

model_to_unlearn_name = "fisher"
model_to_unlearn = copy.deepcopy(original_model)

# import torch.nn as nn
# model_to_unlearn = nn.DataParallel(model_to_unlearn, device_ids=[0, 1])

import time

start_time = time.perf_counter()

unlearned_model = iterative_fisher_unlearn_simplified_golatkar(
    model_to_unlearn,
    criterion,
    rep_subset,
    unlearn_relative_indices,
    SIGMA,
    deletion_batch_size=MINI_BATCH_SIZE,
    compute_batch_size=COMPUTE_BATCH_SIZE,
    eps=EPS,
)

end_time = time.perf_counter()  # End timer
elapsed_time = end_time - start_time

print(f"Execution time: {elapsed_time:.6f} seconds")

## Unlearning via Influence x Lissa


In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm, trange
from torch.amp import GradScaler, autocast
import math

scaler = GradScaler()


def compute_gradient_on_subset(model, criterion, dataset_subset, batch_size):
    """
    Compute the average gradient Δ = (1/|D_u|) Σ_{(x,y) in D_u} ∇_θ L(θ, (x,y))
    over the dataset_subset.
    """
    dataloader = DataLoader(dataset_subset, batch_size=batch_size, shuffle=False)

    grad_dict = {
        name: torch.zeros_like(param, device=DEVICE)
        for name, param in model.named_parameters()
        if param.requires_grad
    }

    total_samples = 0

    model.eval()  # ensure gradients are computed
    for inputs, targets in tqdm(dataloader, desc="Computing gradients"):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        model.zero_grad()

        # scaler.scale(loss).backward()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        current_batch = inputs.size(0)
        total_samples += current_batch

        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                grad_dict[name] += param.grad.detach().clone() * current_batch

    # Average over total samples
    for name in grad_dict:
        grad_dict[name] /= total_samples

    # Flatten all gradients into one vector
    grad_vector = torch.cat(
        [grad_dict[name].view(-1) for name in sorted(grad_dict.keys())]
    )
    return grad_vector


def lissa_inverse_hvp(
    model, criterion, data_loader, v, damping=1e-4, scale=100, recursion_depth=20
):
    convergence_threshold = 1e-4
    ihvp_estimate = v.clone().to(DEVICE)
    ihvp_prev = torch.zeros_like(ihvp_estimate)
    data_iter = iter(data_loader)
    model.eval()

    # Lists to store metrics
    norms_history = []
    delta_norms_history = []
    relative_change_history = []

    for step in trange(recursion_depth, desc="LiSSA iterations", leave=False):
        if step > 0:
            ihvp_prev = ihvp_estimate.clone()
        try:
            inputs, targets = next(data_iter)
        except StopIteration:
            data_iter = iter(data_loader)
            inputs, targets = next(data_iter)

        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        model.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        grads = torch.autograd.grad(loss, model.parameters(), create_graph=True)
        # Flatten gradients
        grad_vector = torch.cat(
            [
                (
                    g.contiguous().view(-1)
                    if g is not None
                    else torch.zeros(p.numel(), device=DEVICE)
                )
                for p, g in zip(model.parameters(), grads)
            ]
        )
        if torch.isnan(grad_vector).any() or torch.isinf(grad_vector).any():
            return torch.zeros_like(v)  

        hv = torch.autograd.grad(
            torch.dot(grad_vector, ihvp_estimate.detach()),
            model.parameters(),
            retain_graph=False,
        )
        # Flatten HVP
        hv_vector = torch.cat(
            [
                (
                    h.contiguous().view(-1)
                    if h is not None
                    else torch.zeros(p.numel(), device=DEVICE)
                )
                for p, h in zip(model.parameters(), hv)
            ]
        )
        if torch.isnan(hv_vector).any() or torch.isinf(hv_vector).any():
            return torch.zeros_like(v)

        # --- Update the estimate ---
        ihvp_estimate = v + (1 - damping) * ihvp_estimate - hv_vector / scale
        if torch.isnan(ihvp_estimate).any() or torch.isinf(ihvp_estimate).any():
            return torch.zeros_like(v)  

        # --- Calculate and Store Metrics ---
        current_norm = torch.norm(ihvp_estimate).item()
        delta_norm = torch.norm(ihvp_estimate - ihvp_prev).item() if step > 0 else 0.0
        prev_norm = torch.norm(ihvp_prev).item() if step > 0 else 0.0
        relative_change = (
            delta_norm / (prev_norm + 1e-9) if prev_norm > 1e-9 else delta_norm
        )

        norms_history.append(current_norm)
        if step > 0:
            delta_norms_history.append(delta_norm)
            relative_change_history.append(relative_change)

    # --- Plotting Section ---
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 3, 1)
    plt.plot(range(recursion_depth), norms_history)
    plt.title("Estimate Norm vs. Iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Norm")
    plt.yscale("log")

    plt.subplot(1, 3, 2)
    plt.plot(range(1, recursion_depth), delta_norms_history)
    plt.title("Absolute Change Norm vs. Iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Norm of Change")
    plt.yscale("log")

    plt.subplot(1, 3, 3)
    plt.plot(range(1, recursion_depth), relative_change_history)
    plt.title("Relative Change vs. Iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Relative Change")
    plt.yscale("log")

    plt.tight_layout()
    plt.show()

    final_norm = norms_history[-1] if norms_history else 0.0
    print(f"Final Estimate Norm: {final_norm:.4e}")
    if math.isnan(final_norm) or math.isinf(final_norm):
        print(f"🚨 ERROR: Final LiSSA estimate norm is NaN/Inf!")
        return torch.zeros_like(v)
    else:
        print("✅ [Debug] LiSSA estimation stable.")
        return ihvp_estimate / scale


# -----------------------------
# Main Function: Iterative Influence Unlearning
# -----------------------------
def iterative_influence_unlearn(
    model,
    criterion,
    full_dataset,
    removal_indices,
    deletion_batch_size,
    compute_batch_size,
    eps,
    max_norm,
    scale=100,
    lissa_depth=50,
):
    """
    Implements iterative Influence Unlearning:
      For each mini-batch of deletion samples, compute the average gradient
      Δ_u, solve v ≈ H⁻¹ Δ_u using Conjugate Gradient on the remaining data,
      and update the model as: θ ← θ + v.

    Mathematical Equations:
      Δ_u = (1/|D_u^i|) Σ_{(x,y) in D_u^i} ∇_θ L(θ, (x,y))
      v ≈ H⁻¹ Δ_u,  where H = ∇²_θ L(θ, D \ D_u^i)
      θ ← θ + v
    """
    full_size = len(full_dataset)
    current_indices = set(range(full_size))

    # Partition removal_indices into mini-batches
    removal_list = list(removal_indices)
    num_batches = math.ceil(len(removal_list) / deletion_batch_size)
    partitioned_removals = [
        removal_list[i * deletion_batch_size : (i + 1) * deletion_batch_size]
        for i in range(num_batches)
    ]
    print(
        f"Total deletion samples: {len(removal_list)}; partitioned into {num_batches} mini-batches (each up to {deletion_batch_size} samples)."
    )

    for i, batch in enumerate(tqdm(partitioned_removals, desc="Influence Unlearning")):
        # Update remaining indices: D_current ← D \ D_u^i
        current_indices -= set(batch)
        updated_indices = sorted(list(current_indices))
        dataset_remaining = Subset(full_dataset, updated_indices)
        print(
            f"Iteration {i+1}/{num_batches}: Remaining dataset size = {len(dataset_remaining)}"
        )

        # Compute average gradient Δ_u for the deletion mini-batch
        deleted_subset = Subset(full_dataset, batch)
        delta = compute_gradient_on_subset(
            model, criterion, deleted_subset, compute_batch_size
        )

        if torch.isnan(delta).any() or torch.isinf(delta).any():
            print("🚨 [Debug] Gradient delta contains NaN/Inf!")
        else:
            print("✅ [Debug] Gradient delta stable.")

        # Create a DataLoader for remaining data to approximate Hessian
        remaining_loader = DataLoader(
            dataset_remaining, batch_size=compute_batch_size, shuffle=True
        )

        influence_update = lissa_inverse_hvp(
            model,
            criterion,
            remaining_loader,
            delta,
            damping=eps,
            scale=scale,
            recursion_depth=lissa_depth,
        )
        # Optionally clip the update to avoid overly large changes
        update_norm = influence_update.norm(2).item()
        if update_norm > max_norm or math.isnan(update_norm) or math.isinf(update_norm):
            print(
                f"WARNING: Clipping influence update from {update_norm:.2f} to {max_norm}"
            )
            influence_update = influence_update * (max_norm / update_norm)
        print(
            f"Iteration {i+1}: Influence update norm = {influence_update.norm(2).item():.4f}"
        )

        # Update model parameters: θ ← θ + v
        pointer = 0
        with torch.no_grad():
            for name, param in model.named_parameters():
                if param.requires_grad:
                    numel = param.numel()
                    update_segment = influence_update[
                        pointer : pointer + numel
                    ].view_as(param)
                    param.data = param.data + update_segment
                    pointer += numel
        print(f"Iteration {i+1}/{num_batches} update completed.")

    return model

In [ ]:
# COMPUTE_BATCH_SIZE = 4
# MINI_BATCH_SIZE = 25

# EPS = 1e-6
# MAX_NORM = 5 # 1, 5, 10
# LISSA_DEPTH = 1000
# SCALE = 10000

In [ ]:
# import copy
# model_to_unlearn_name = 'influence'
# model_to_unlearn = copy.deepcopy(original_model)

# # import torch.nn as nn
# # model_to_unlearn = nn.DataParallel(model_to_unlearn, device_ids=[0, 1])

In [ ]:
# import time
# start_time = time.perf_counter()


# unlearned_model = iterative_influence_unlearn(model_to_unlearn, criterion, rep_subset, unlearn_relative_indices,
#                                                   MINI_BATCH_SIZE, COMPUTE_BATCH_SIZE,
#                                                   EPS, MAX_NORM, SCALE, LISSA_DEPTH)

# end_time = time.perf_counter()  # End timer
# elapsed_time = end_time - start_time

# print(f"Execution time: {elapsed_time:.6f} seconds")

In [ ]:
save_model(unlearned_model, f"{model_to_unlearn_name}_model.pth")

## Test unlearned model


In [ ]:
model, *_ = init_model_resnet50()
model = nn.DataParallel(model, device_ids=[0, 1])
model_path = f"{model_to_unlearn_name}_model.pth"

In [ ]:
test_model(model, model_to_unlearn_name, model_path, test_loader)

In [ ]:
predictions_path = f"{model_to_unlearn_name}_predictions.json"
# classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
show_metrics(predictions_path, classes, model_to_unlearn_name)

# Fine-Tuning on Shadow Twins


In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

gold_train_subset = gold_train_loader.dataset

fine_tune_results = {}
for method_name, twin_mapping, similarity_metrics in twin_mappings_list:
    print(f"\n=== Fine-tuning using replacement method: {method_name} ===")

    twin_indices = list(twin_mapping.values())
    twin_subset = Subset(train_dataset_full, twin_indices)
    combined_train_dataset = ConcatDataset([gold_train_subset, twin_subset])
    fine_tune_loader = DataLoader(
        combined_train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )

    ft_model_name = f"fine_tuned_{model_to_unlearn_name}_on_{method_name}"
    model_ft = copy.deepcopy(unlearned_model)

    LR_finetune = 1e-3
    # optimizer_ft = optim.AdamW(model_ft.parameters(), lr=LR_finetune, weight_decay=1e-4)
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=LR_finetune, weight_decay=0)

    start_time = time.perf_counter()
    train_model(
        model_ft,
        ft_model_name,
        fine_tune_loader,
        gold_valid_loader,
        criterion,
        optimizer_ft,
        num_epochs=1,
    )
    end_time = time.perf_counter()
    training_time = end_time - start_time
    print(f"Training time for {ft_model_name}: {training_time:.6f} seconds")

    # --- Evaluate Fine-Tuned Model ---
    model, *_ = init_model_resnet50()
    model = nn.DataParallel(model, device_ids=[0, 1])
    test_model(model, ft_model_name, f"{ft_model_name}_model.pth", gold_test_loader)

    predictions_path = f"{ft_model_name}_predictions.json"
    show_metrics(predictions_path, classes, ft_model_name)

print(f"\n=== Fine-tuning using remaining set ===")
ft_model_name = f"fine_tuned_{model_to_unlearn_name}_on_remaining"
model_ft = copy.deepcopy(unlearned_model)

LR_finetune = 1e-3
optimizer_ft = optim.Adam(model_ft.parameters(), lr=LR_finetune, weight_decay=0)
scheduler_ft = StepLR(optimizer_ft, step_size=2, gamma=0.5)

start_time = time.perf_counter()
train_model(
    model_ft,
    ft_model_name,
    gold_train_loader,
    gold_valid_loader,
    criterion,
    optimizer_ft,
    num_epochs=1,
)
end_time = time.perf_counter()
training_time = end_time - start_time
print(f"Training time for {ft_model_name}: {training_time:.6f} seconds")

# --- Evaluate Fine-Tuned Model ---
model, *_ = init_model_resnet50()
model = nn.DataParallel(model, device_ids=[0, 1])
test_model(model, ft_model_name, f"{ft_model_name}_model.pth", gold_test_loader)

predictions_path = f"{ft_model_name}_predictions.json"
show_metrics(predictions_path, classes, ft_model_name)

In [ ]:
import torch
import json
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score  # Import accuracy_score
from utils.utils import DEVICE  # Assuming DEVICE is defined in your utils


def test_model(model, model_name, model_path, test_loader):
    """
    Loads model state, evaluates on the test loader, saves predictions,
    and returns the test accuracy.

    Args:
        model (torch.nn.Module): The model instance (architecture should match the state dict).
        model_name (str): A name for saving predictions and logging.
        model_path (str or Path): Path to the saved model state dictionary (.pth file).
        test_loader (DataLoader): DataLoader for the test set.

    Returns:
        float: The calculated accuracy score on the test set.
    """
    print(f"Loading and testing model: {model_name} from {model_path}")
    try:
        # Load state dict into the provided model object
        model.load_state_dict(
            torch.load(model_path, weights_only=True, map_location=DEVICE)
        )
        # model = nn.DataParallel(model, device_ids=[0, 1])
        # # model.to(DEVICE)
        print("Model state loaded successfully.")
    except Exception as e:
        print(f"Error loading model state dict from {model_path}: {e}")
        return 0.0

    model.eval()

    predictions = []
    true_labels = []

    with torch.inference_mode():
        for inputs, labels in tqdm(test_loader, desc=f"Evaluating {model_name}"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Ensure lists are flat if model outputs batches > 1 consistently
    predictions = np.array(predictions).flatten()
    true_labels = np.array(true_labels).flatten()

    # --- Calculate Accuracy ---
    if len(true_labels) > 0:
        accuracy = accuracy_score(true_labels, predictions)
        print(f"Test Accuracy for {model_name}: {accuracy:.4f}")
    else:
        print("Warning: Test loader was empty. Accuracy is 0.")
        accuracy = 0.0

    results = {"predictions": predictions.tolist(), "true_labels": true_labels.tolist()}
    pred_file = f"{model_name}_predictions.json"
    try:
        with open(pred_file, "w") as f:
            json.dump(results, f)
        print(f"Predictions and labels saved to {pred_file}")
    except Exception as e:
        print(f"Error saving predictions to {pred_file}: {e}")

    # --- Return Accuracy ---
    return float(accuracy)


RUN_DIR = Path("./unlearning_runs")
RUN_DIR.mkdir(exist_ok=True)
NUM_RUNS = 5
gold_standard_accuracy = 0.9097
print(f"Using Gold Standard Accuracy: {gold_standard_accuracy:.4f}")

In [ ]:
# model_file = f"{model_name}_model.pth"
model_file = f"/kaggle/input/ul-resnet/pytorch/default/3/base_model.pth"
original_model, original_model_name, criterion, _optimizer, transform = (
    load_model_resnet50(model_pth_path=model_file, parallel=True)
)

In [ ]:
import time

fisher_accuracies = []
fisher_model_paths = []
fisher_times = []
influence_accuracies = []
influence_model_paths = []
influence_times = []

# --- Fisher Runs ---
print(f"\n--- Running Fisher Unlearning ({NUM_RUNS} runs) ---")
for i in range(NUM_RUNS):
    run_name = f"fisher_run_{i+1}"
    print(f"  Fisher Run {i+1}/{NUM_RUNS} ({run_name})")

    # Load fresh model instance and state dict for each run
    model_to_unlearn, *_ = init_model_resnet50()
    model_to_unlearn = copy.deepcopy(original_model)
    # model_to_unlearn = nn.DataParallel(model_to_unlearn, device_ids=[0, 1])
    # model_to_unlearn.to(DEVICE)

    start_time = time.perf_counter()
    # Perform unlearning
    unlearned_model_fisher = iterative_fisher_unlearn_simplified_golatkar(
        model=model_to_unlearn,
        criterion=criterion,
        full_dataset=rep_subset,
        removal_indices=unlearn_relative_indices,
        sigma_lambda_term=35e-6,
        deletion_batch_size=25,
        compute_batch_size=128,
        eps=1e-6,
    )
    end_time = time.perf_counter()
    duration = end_time - start_time
    fisher_times.append(duration)
    print(f"  Fisher Run {i+1} Time: {duration:.2f} seconds")
    model_path = RUN_DIR / f"{run_name}.pth"
    save_model(unlearned_model_fisher, model_path)

    # Test the unlearned model
    eval_model, *_ = init_model_resnet50()
    eval_model = nn.DataParallel(eval_model, device_ids=[0, 1])
    accuracy = test_model(eval_model, run_name, model_path, gold_test_loader)
    fisher_accuracies.append(accuracy)
    fisher_model_paths.append(model_path)
    print(f"  Fisher Run {i+1} Accuracy: {accuracy:.4f}")

    # Cleanup
    del model_to_unlearn, unlearned_model_fisher, eval_model
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# --- Influence Runs ---
print(f"\n--- Running Influence Unlearning ({NUM_RUNS} runs) ---")
for i in range(NUM_RUNS):
    run_name = f"influence_run_{i+1}"
    print(f"  Influence Run {i+1}/{NUM_RUNS} ({run_name})")

    # Load fresh model instance and state dict
    model_to_unlearn, *_ = init_model_resnet50()
    model_to_unlearn = copy.deepcopy(original_model)
    # model_to_unlearn = nn.DataParallel(model_to_unlearn, device_ids=[0, 1])
    # model_to_unlearn.to(DEVICE)

    start_time = time.perf_counter()

    # Perform unlearning
    unlearned_model_influence = iterative_influence_unlearn(
        model=model_to_unlearn,
        criterion=criterion,
        full_dataset=rep_subset,
        removal_indices=unlearn_relative_indices,
        deletion_batch_size=25,
        compute_batch_size=4,
        eps=1e-6,
        max_norm=5,
        scale=10000,
        lissa_depth=1000,
    )

    end_time = time.perf_counter()
    duration = end_time - start_time
    influence_times.append(duration)

    print(f"  Fisher Run {i+1} Time: {duration:.2f} seconds")
    model_path = RUN_DIR / f"{run_name}.pth"
    save_model(unlearned_model_influence, model_path)

    # Test
    eval_model, *_ = init_model_resnet50()
    eval_model = nn.DataParallel(eval_model, device_ids=[0, 1])
    accuracy = test_model(eval_model, run_name, model_path, gold_test_loader)
    influence_accuracies.append(accuracy)
    influence_model_paths.append(model_path)
    print(f"  Influence Run {i+1} Accuracy: {accuracy:.4f}")

    # Cleanup
    del model_to_unlearn, unlearned_model_influence, eval_model
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# %%
# =============================================================================
# Phase 2: Analyze Unlearning Variability & Select Representative Models
# =============================================================================

fisher_accuracies = np.array(fisher_accuracies)
influence_accuracies = np.array(influence_accuracies)
fisher_times = np.array(fisher_times)
influence_times = np.array(influence_times)

fisher_mean_acc = np.mean(fisher_accuracies)
fisher_std_acc = np.std(fisher_accuracies)
fisher_var_acc = np.var(fisher_accuracies)
fisher_mean_time = np.mean(fisher_times)

influence_mean_acc = np.mean(influence_accuracies)
influence_std_acc = np.std(influence_accuracies)
influence_var_acc = np.var(influence_accuracies)
influence_mean_time = np.mean(influence_times)

print("\n--- Unlearning Method Stats ---")
print(
    f"Fisher   | Mean Acc: {fisher_mean_acc:.4f}, StdDev Acc: {fisher_std_acc:.4f}, Var Acc: {fisher_var_acc:.4f} | Mean Time: {fisher_mean_time:.2f}s"
)
print(
    f"Influence| Mean Acc: {influence_mean_acc:.4f}, StdDev Acc: {influence_std_acc:.4f}, Var Acc: {influence_var_acc:.4f} | Mean Time: {influence_mean_time:.2f}s"
)

fisher_median_idx = np.argmin(np.abs(fisher_accuracies - fisher_mean_acc))
influence_median_idx = np.argmin(np.abs(influence_accuracies - influence_mean_acc))

fisher_rep_model_path = fisher_model_paths[fisher_median_idx]
influence_rep_model_path = influence_model_paths[influence_median_idx]

fisher_rep_acc = fisher_accuracies[fisher_median_idx]
influence_rep_acc = influence_accuracies[influence_median_idx]

print(
    f"\nSelected Representative Fisher Model: {fisher_rep_model_path.name} (Acc: {fisher_rep_acc:.4f})"
)
print(
    f"Selected Representative Influence Model: {influence_rep_model_path.name} (Acc: {influence_rep_acc:.4f})"
)

In [ ]:
# Define Epoch settings for healing
HEAL_BATCH_SIZE = BATCH_SIZE
HEAL_EPOCHS_SHORT = 1
HEAL_EPOCHS_LONG = 3
HEAL_LR = 0.001
epoch_settings_to_run = [HEAL_EPOCHS_SHORT, HEAL_EPOCHS_LONG]

# Results dictionary to store final accuracies
results = {"fisher_start": {}, "influence_start": {}}

# Define the starting models
start_points = {
    "fisher_start": fisher_rep_model_path,
    "influence_start": influence_rep_model_path,
}

# --- Define the 5 Healing Data Loaders/Configurations ---
healing_data_configs = []
gold_train_subset = gold_train_loader.dataset
# Configs 1-4: FT-RemainPlusTwins
if "twin_mappings_list" in locals() and twin_mappings_list:
    print("--- Defining Remain+Twins Healing Loaders ---")
    for twin_method_name, twin_mapping, _ in twin_mappings_list:
        print(f"  Processing twin method: {twin_method_name}")
        try:
            if not isinstance(twin_mapping, dict) or not twin_mapping:
                print(f"    Skipping {twin_method_name}: invalid mapping.")
                continue
            twin_indices = list(twin_mapping.values())
            if twin_indices:
                twin_subset = Subset(train_dataset_full, twin_indices)
                if not isinstance(gold_train_subset, torch.utils.data.Dataset):
                    print("    Error: gold_train_subset is not a Dataset object.")
                    continue
                combined_train_dataset = ConcatDataset([gold_train_subset, twin_subset])
                heal_loader = DataLoader(
                    combined_train_dataset,
                    batch_size=HEAL_BATCH_SIZE,
                    shuffle=True,
                    num_workers=4,
                    pin_memory=True,
                )
                healing_data_configs.append(
                    {
                        "name": f"RemainPlusTwins-{twin_method_name}",
                        "loader": heal_loader,
                    }
                )
                print(f"    Loader created for {twin_method_name}.")
            else:
                print(f"    Skipping {twin_method_name}: No twin indices.")
        except Exception as e:
            print(f"    Error creating loader for {twin_method_name}: {e}")
else:
    print(
        "Warning: twin_mappings_list not found or empty. Cannot create twin-based healing loaders."
    )

# Config 5: FT-RemainOnly
print("--- Defining RemainOnly Healing Loader ---")
healing_data_configs.append(
    {
        "name": "RemainOnly",
        "loader": gold_train_loader,  # Use the pre-defined loader for D \ D_u
    }
)
print(f"Defined {len(healing_data_configs)} healing data configurations.")

# --- Iterate through starting models, healing datasets, and epoch settings ---
for start_point_name, start_model_path in start_points.items():
    print(
        f"\n\n{'='*20} Processing Start Point: {start_point_name} ({start_model_path.name}) {'='*20}"
    )

    # Load the representative model state dict ONCE for this starting point
    start_model, *_ = load_model_resnet50(start_model_path, parallel=True)
    # start_model_state_dict = torch.load(start_model_path, weights_only=True, map_location=DEVICE)
    initial_accuracy = (
        fisher_rep_acc if start_point_name == "fisher_start" else influence_rep_acc
    )
    print(f"--- Initial Accuracy for {start_point_name}: {initial_accuracy:.4f} ---")

    # Initialize nested dictionary for this start point
    results[start_point_name] = {config["name"]: {} for config in healing_data_configs}

    # Iterate through the 5 healing data configurations
    for config in healing_data_configs:
        healing_config_name = config["name"]
        healing_loader = config["loader"]
        print(f"\n  --- Applying Healing using Data: {healing_config_name} ---")

        # Iterate through the epoch settings (1 and 5 epochs)
        for num_epochs in epoch_settings_to_run:
            print(f"    --- Healing for {num_epochs} epoch(s) ---")
            try:
                # Load fresh model instance and the STARTING state dict EVERY time
                model_to_heal, *_ = init_model_resnet50()
                model_to_heal = copy.deepcopy(start_model)
                # model_to_heal = nn.DataParallel(model_to_heal, device_ids=[0, 1])
                model_to_heal.to(DEVICE)

                # Define optimizer
                current_lr = HEAL_LR
                optimizer_heal = optim.Adam(
                    model_to_heal.parameters(), lr=current_lr, weight_decay=0
                )

                # Define healed model name including epoch count
                healed_model_name = f"healed_{start_point_name}_using_{healing_config_name}_{num_epochs}ep"

                # Run Healing (Fine-tuning)
                start_heal_time = time.perf_counter()
                train_model(
                    model=model_to_heal,
                    model_name=healed_model_name,
                    train_loader=healing_loader,
                    val_loader=gold_valid_loader,
                    criterion=criterion,
                    optimizer=optimizer_heal,
                    num_epochs=num_epochs,
                )
                end_heal_time = time.perf_counter()
                print(f"      Healing time: {end_heal_time - start_heal_time:.2f}s")

                # Save the healed model
                healed_model_path = RUN_DIR / f"{healed_model_name}.pth"
                save_model(model_to_heal, healed_model_path)

                # Test the healed model
                eval_model, *_ = init_model_resnet50()
                eval_model = nn.DataParallel(eval_model, device_ids=[0, 1])
                accuracy = test_model(
                    eval_model, healed_model_name, healed_model_path, gold_test_loader
                )
                results[start_point_name][healing_config_name][num_epochs] = accuracy
                print(f"      Healed Accuracy ({num_epochs}ep): {accuracy:.4f}")

                # Cleanup
                del model_to_heal, optimizer_heal, eval_model
                torch.cuda.empty_cache()
                gc.collect()

            except Exception as e:
                print(
                    f"      ERROR during healing ({healing_config_name}, {num_epochs}ep): {e}"
                )
                if "model_to_heal" in locals():
                    del model_to_heal
                if "optimizer_heal" in locals():
                    del optimizer_heal
                if "eval_model" in locals():
                    del eval_model
                torch.cuda.empty_cache()
                gc.collect()

In [ ]:
print("\n\n--- FINAL RESULTS SUMMARY ---")
print(f"Gold Standard Accuracy: {gold_standard_accuracy:.4f}")

# Define the epoch counts we analyzed
epoch_settings_analyzed = [HEAL_EPOCHS_SHORT, HEAL_EPOCHS_LONG]  # e.g., [1, 5]

for start_point_name, start_acc in [
    ("fisher_start", fisher_rep_acc),
    ("influence_start", influence_rep_acc),
]:
    print(f"\n\n{'='*10} Healing Summary starting from: {start_point_name} {'='*10}")
    print(f"  Initial Accuracy: {start_acc:.4f}")

    healing_results_for_start = results.get(start_point_name, {})
    if not healing_results_for_start:
        print("  No healing results found for this starting point.")
        continue

    # Analyze results separately for each epoch setting
    for num_epochs in epoch_settings_analyzed:
        print(f"\n  --- Analysis for {num_epochs}-Epoch Healing ---")

        epoch_specific_accs = {}
        # Extract accuracies only for the current number of epochs
        for config_name, epoch_results in healing_results_for_start.items():
            if num_epochs in epoch_results:
                # Store accuracy with the config name as the key
                epoch_specific_accs[config_name] = epoch_results[num_epochs]

        if not epoch_specific_accs:
            print(f"    No results found for {num_epochs}-epoch healing.")
            continue

        # Find best method/accuracy for this epoch count
        best_method_this_epoch = max(epoch_specific_accs, key=epoch_specific_accs.get)
        best_acc_this_epoch = epoch_specific_accs[best_method_this_epoch]
        pp_diff_best = (best_acc_this_epoch - gold_standard_accuracy) * 100

        # Find worst method/accuracy for this epoch count
        worst_method_this_epoch = min(epoch_specific_accs, key=epoch_specific_accs.get)
        worst_acc_this_epoch = epoch_specific_accs[worst_method_this_epoch]
        pp_diff_worst = (worst_acc_this_epoch - gold_standard_accuracy) * 100

        print(
            f"    Best  Accuracy ({num_epochs}ep): {best_acc_this_epoch:.4f} (using {best_method_this_epoch}) | Diff to Gold: {pp_diff_best:+.2f} pp"
        )
        print(
            f"    Worst Accuracy ({num_epochs}ep): {worst_acc_this_epoch:.4f}                            | Diff to Gold: {pp_diff_worst:+.2f} pp"
        )

print("\n--- End of Summary ---")